In [1]:
import plaidml.keras
import os
plaidml.keras.install_backend()

In [2]:
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [4]:
import keras
import keras.backend as K

K

<module 'plaidml.keras.backend' from 'C:\\Users\\Bamouh Mohamed\\AppData\\Local\\conda\\conda\\envs\\Projectenv\\lib\\site-packages\\plaidml\\keras\\backend.py'>

In [5]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from collections import Counter
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
import cv2
from itertools import chain
from pynput.keyboard import Key, Controller
from keras.models import model_from_json
import glob

In [6]:
kernel = np.ones((3,3),np.uint8)

In [7]:
pathTest = "fingers/test/*"
NUMBER_OF_CLASSES = 6
#indice_to_label = {0:"0L", 1:"1L", 2:"2L", 3:"3L", 4:"4L", 5:"5L", 6:"0R", 7:"1R", 8:"2R", 9:"3R", 10:"4R", 11:"5R"}
indice_to_label = {0:"0", 1:"1", 2:"2", 3:"3", 4:"4", 5:"5"}
label_to_indice = {v:k for k,v in indice_to_label.items()}

In [8]:
lower = np.array([0, 48, 80], dtype = "uint8")
upper = np.array([20, 255, 255], dtype = "uint8")

lower_gray = np.array([42], dtype = "uint8")
upper_gray = np.array([176], dtype = "uint8")

In [9]:
def process_image_from_video(img):
    img = cv2.resize(img, (128, 128))
    img = cv2.GaussianBlur(img,(5,5),0)
    _,img = cv2.threshold(img,20,255,cv2.THRESH_BINARY)
    #kernel = np.ones((3,3),np.uint8)
    #img = cv2.dilate(img, kernel, iterations = 1)
    #img = cv2.erode(img, kernel, iterations = 1)
    img = img/255
    return img

def process_image_for_training(img):
    img = cv2.resize(img, (128, 128))
    img = cv2.GaussianBlur(img,(5,5),0)
    _,img = cv2.threshold(img,80,255,cv2.THRESH_BINARY)
    img = img/255
    return img

def img_to_sample(img):
    img = img.astype(np.uint8)
    img = np.reshape(img, (128, 128))
    
    img = process_image_for_training(img)
    
    img = np.reshape(img, (128, 128, 1))
    return img

def loadModel(architecturePath="handmodel_fingers_architecture.json", weightsPath="handmodel_fingers_weights.hdf5"):
    # Model reconstruction from JSON file
    with open(architecturePath, 'r') as f:
        model = model_from_json(f.read())

    # Load weights into the new model
    model.load_weights(weightsPath)

    return model

In [39]:
tf.test.gpu_device_name()

''

In [10]:
model = loadModel(architecturePath="handmodel_fingers_architecture.json", weightsPath="handmodel_fingers_weights.hdf5")

INFO:plaidml:Opening device "opencl_amd_iceland.0"


In [34]:
cap = cv2.VideoCapture(0)
keyboard = Controller()

lower = np.array([0, 48, 80], dtype = "uint8")
upper = np.array([20, 255, 255], dtype = "uint8")
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.flip(frame,1)
    
    left_zone = gray[100:350, 10:260, :]
    cv2.rectangle(gray,(10,100),(260,350),(0,255,0),3)
    
    right_zone = gray[100:350, 400:650, :]
    cv2.rectangle(gray,(400,100),(650,350),(0,255,0),3)
    
    left_zone = cv2.cvtColor(left_zone, cv2.COLOR_BGR2HSV)
    right_zone = cv2.cvtColor(right_zone, cv2.COLOR_BGR2HSV)
    
    cv2.imshow('HSV left',left_zone)
    cv2.imshow('HSV right',right_zone)
    
    skinMask_left = cv2.inRange(left_zone, lower, upper)
    skinMask_right = cv2.inRange(right_zone, lower, upper)
    
    skinMask_left = cv2.dilate(skinMask_left, kernel, iterations = 2)
    skinMask_left = cv2.erode(skinMask_left, kernel, iterations = 2)
    skinMask_left = cv2.GaussianBlur(skinMask_left, (3, 3), 0)
    
    skinMask_right = cv2.dilate(skinMask_right, kernel, iterations = 2)
    skinMask_right = cv2.erode(skinMask_right, kernel, iterations = 2)
    skinMask_right = cv2.GaussianBlur(skinMask_right, (3, 3), 0)
    
    left_zone = cv2.bitwise_and(left_zone, left_zone, mask = skinMask_left)
    right_zone = cv2.bitwise_and(right_zone, right_zone, mask = skinMask_right)
       
    #cv2.rectangle(gray,(10,100),(138,228),(0,255,0),3)
    
    # Capture the hand zone
    #zone = gray[100:228, 10:138]
    #zone = gray[100:350, 10:260, :]
    left_zone = cv2.cvtColor(left_zone, cv2.COLOR_BGR2GRAY)
    right_zone = cv2.cvtColor(right_zone, cv2.COLOR_BGR2GRAY)
    
    left_zone = process_image_from_video(left_zone)
    right_zone = process_image_from_video(right_zone)
    
    left_zone_pred = left_zone.reshape((1, 128, 128, 1))
    right_zone_pred = right_zone.reshape((1, 128, 128, 1))
    
    
    left_zone_pred = model.predict(left_zone_pred)
    right_zone_pred = model.predict(right_zone_pred)
    
    #categorical
    predicted_class_left = np.argmax(left_zone_pred)
    predicted_class_right = np.argmax(right_zone_pred)
    
    labels_left = ["{} : {}%".format(k, format(left_zone_pred[0][v]*100, '.2f')) for k,v in label_to_indice.items()]
    labels_right = ["{} : {}%".format(k, format(right_zone_pred[0][v]*100, '.2f')) for k,v in label_to_indice.items()]
    
    if left_zone[64,64] == 255:
        #keyboard.press(Key.left) 
        None
    elif right_zone[64,64] == 255:
        #keyboard.press(Key.right)
        None
    else:
        #keyboard.release(Key.right)
        #keyboard.release(Key.left)
        None
        
    if predicted_class_left == 0 or predicted_class_right == 0:
        #keyboard.press(Key.space)
        None
    else:
        #keyboard.release(Key.space)
        None
    
    cv2.imshow('What the Network Sees left',left_zone)
    cv2.imshow('What the Network Sees right',right_zone)
    
    y = 100
    cv2.putText(gray,indice_to_label[int(predicted_class_left)],(270,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2,cv2.LINE_AA)
    for label in labels_left:
        y = y + 20
        cv2.putText(gray, label,(270, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2,cv2.LINE_AA)
        
    yy = 250
    cv2.putText(gray,indice_to_label[int(predicted_class_right)],(270,yy), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2,cv2.LINE_AA)
    for label in labels_right:
        yy = yy + 20
        cv2.putText(gray, label,(270, yy), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2,cv2.LINE_AA)

    # Display the resulting frame      
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break                                                                                                                     

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()